# First Stage Modeling

In [1]:
from models.config import import_config
from data.datasets import GeneralDataset
from models.main import *

In [2]:
args = ModelArgs()

## Import last config

In [3]:
config = import_config()

## Import data and create criterion

In [4]:
from torch.utils.data import random_split, DataLoader
"""Hyperparameter"""
batch_size = 32

#Main Dataset
dataset = GeneralDataset(*args.first_stage_config.values())
#Splitting on training and validation data
# 80/20
train_len = round(0.8*len(dataset))
val_len = len(dataset) - train_len
train_ds, val_ds = random_split(dataset, [train_len, val_len])
# Dataloader
train_dl = DeviceDataLoader(DataLoader(train_ds, batch_size, True, pin_memory=True, num_workers=4), config.device)
val_dl = DeviceDataLoader(DataLoader(val_ds, 2*batch_size, False, pin_memory=True, num_workers=4), config.device)

Processing:    0%|          |  [ Elapsed: 00:00, Remaining: ? ] [1/1] 

Downloading:   0%|          |  [ Elapsed: 00:00, Remaining: ? ] (91.611MB)

KeyboardInterrupt: 

In [ ]:
criterion = PretrainingCriterion(train_dl)

## Defining model

In [4]:
encoder = RnnEncoder()
decoder = RnnDecoder()

model = Model(encoder, decoder, config, criterion, args)

TypeError: RnnEncoder.__init__() missing 1 required positional argument: 'args'

In [ ]:
"""Hyperparameters"""
lr = 1e-2
weight_decay = 1e-4
grad_clip = 5e-2
opt_func = torch.optim.Adam
lr_sched = torch.optim.lr_scheduler.OneCycleLR
encoder_forcing = True
weights = [0.1,0.2,0.7]

In [ ]:
model.fit(train_dl, val_dl, lr, weight_decay, grad_clip, opt_func, lr_sched, encoder_forcing, weights)